In [1]:
import pandas as pd
import os
from dotenv import load_dotenv
import json
import re
import requests
import time

import swagger_client
from swagger_client.rest import ApiException

In [2]:
requests.packages.urllib3.util.ssl_.DEFAULT_CIPHERS += 'HIGH:!DH:!aNULL'
try:
    requests.packages.urllib3.contrib.pyopenssl.DEFAULT_SSL_CIPHER_LIST += 'HIGH:!DH:!aNULL'
except AttributeError:
    # no pyopenssl support used / needed / available
    pass

In [3]:
load_dotenv()
AEMET_KEY = os.getenv("KEY_AEMET")
configuration = swagger_client.Configuration()
configuration.api_key['api_key'] = AEMET_KEY
api_instance = swagger_client.ValoresClimatologicosApi(swagger_client.ApiClient(configuration))

In [25]:
df15 = pd.read_csv('data/Temp15-16_meteo.csv')
df16 = pd.read_csv('data/Temp16-17_meteo.csv')
df17 = pd.read_csv('data/Temp17-18_meteo.csv')

In [20]:
df15.loc[df15.prec=='Ip']

,Equipo Local,Goles,Ataque,Medio,Defensa,Goles_Acumulados,Equipo Visitante,Goles.1,Ataque.1,Medio.1,...,Resultado,cod_est,tmin,tmed,tmax,prec,presmin,presmax,velmedia,rachaviento


In [12]:
est_meteo_mod15 = {

'Las Palmas': {'prov':'Las Palmas',
               'cod_est':'C649I'},

}

In [13]:
df15.loc[df15.tmin.isnull(), 'cod_est'] = df15.loc[df15.tmin.isnull()]['Equipo Local'].apply(lambda x: est_meteo_mod15[x]['cod_est'])

In [15]:
for index, row in df15.loc[df15.tmin.isnull()].iterrows():
    print(f'Obteniendo datos de la fila: {index}')
    fecha_ini = row.Día + 'T00:00:00UTC'
    fecha_fin = row.Día + 'T23:59:59UTC'
    estacion = row.cod_est
    
    try:
        respuesta_api = api_instance.climatologas_diarias_(fecha_ini, fecha_fin, estacion)
        url_respuesta = respuesta_api.datos
        response = requests.request("GET", url_respuesta)
        print(response)
        print('\n\n')
        meteo = json.loads(response.text)[0]
        

        try:
            df15.loc[index, 'tmin'] = meteo['tmin']
        except:
            df15.loc[index, 'tmin'] = '-'
        try:
            df15.loc[index, 'tmed'] = meteo['tmed']
        except:
            df15.loc[index, 'tmed'] = '-'
        try:
            df15.loc[index, 'tmax'] = meteo['tmax']
        except:
            df15.loc[index, 'tmax'] = '-'
        try:
            df15.loc[index, 'prec'] = meteo['prec']
        except:
            df15.loc[index, 'prec'] = '-'
        try:
            df15.loc[index, 'presmin'] = meteo['presMin']
        except:
            df15.loc[index, 'presmin'] = '-'
        try:
            df15.loc[index, 'presmax'] = meteo['presMax']
        except:
            df15.loc[index, 'presmax'] = '-'
        try:
            df15.loc[index, 'velmedia'] = meteo['velmedia']
        except:
            df15.loc[index, 'velmedia'] = '-'
        try:
            df15.loc[index, 'rachaviento'] = meteo['racha']
        except:
            df15.loc[index, 'rachaviento'] = '-'

        time.sleep(5)
    except ApiException as e:
        print ('Algo ha fallado')
    

Obteniendo datos de la fila: 261
<Response [200]>





In [17]:
df15.iloc[261]

Equipo Local          Las Palmas
Goles                          4
Ataque                        73
Medio                         71
Defensa                       70
Goles_Acumulados              25
Equipo Visitante          Getafe
Goles.1                        0
Ataque.1                      73
Medio.1                       75
Defensa.1                     74
Goles_Acumulados.1            26
Día                   2016-03-01
Hora                       22:00
Jornada                       27
Resultado                      2
cod_est                    C649I
tmin                        16,3
tmed                        18,5
tmax                        20,7
prec                         0,0
presmin                   1017,2
presmax                   1020,8
velmedia                     8,3
rachaviento                 15,3
Name: 261, dtype: object

In [23]:
def change_prec(val):
    if val == 'Ip':
        return '0,05'
    elif val == '-':
        return '0,0'
    else:
        return val
    
df15.prec = df15.prec.apply(lambda x: change_prec(x))

In [24]:
df15.to_csv('data/Temp15-16_meteo.csv', index=False)

In [22]:
df15.loc[:,'tmin':].head()

,tmin,tmed,tmax,prec,presmin,presmax,velmedia,rachaviento
0,24.400000,28.000000,31.700001,0.00,1010.400024,1014.299988,2.2,8.1
1,16.200001,19.799999,23.299999,5.80,1004.500000,1010.400024,3.3,13.1
2,23.500000,26.200001,28.799999,0.05,1010.099976,1018.000000,2.2,8.3
3,17.400000,24.600000,31.700001,0.00,934.099976,942.700012,2.5,11.7
4,17.400000,24.600000,31.700001,0.00,934.099976,942.700012,2.5,11.7


In [21]:
df15.loc[:,'tmin':] = df15.loc[:,'tmin':].apply(lambda x: x.str.replace(',', '.'))
df15.loc[:,'tmin':] = df15.loc[:,'tmin':].apply(pd.to_numeric, downcast='float')

In [26]:
df15.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 380 entries, 0 to 379
Data columns (total 25 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Equipo Local        380 non-null    object 
 1   Goles               380 non-null    int64  
 2   Ataque              380 non-null    int64  
 3   Medio               380 non-null    int64  
 4   Defensa             380 non-null    int64  
 5   Goles_Acumulados    380 non-null    int64  
 6   Equipo Visitante    380 non-null    object 
 7   Goles.1             380 non-null    int64  
 8   Ataque.1            380 non-null    int64  
 9   Medio.1             380 non-null    int64  
 10  Defensa.1           380 non-null    int64  
 11  Goles_Acumulados.1  380 non-null    int64  
 12  Día                 380 non-null    object 
 13  Hora                380 non-null    object 
 14  Jornada             380 non-null    int64  
 15  Resultado           380 non-null    int64  
 16  cod_est 

In [ ]:
est_meteo = {
'Eibar': {'prov':'Gipuzkoa',
         'cod_est':'1050J'},
'Granada': {'prov':'Granada',
            'cod_est':'5530E'},
'Sporting de Gijón': {'prov':'Asturias',
                      'cod_est':'1207U'},
'Girona': {'prov':'Girona',
           'cod_est':'0367'},
'Espanyol': {'prov':'Barcelona',
             'cod_est':'0076'},
'Rayo Vallecano': {'prov':'Madrid',
                   'cod_est':'3195'},
'Athletic Club': {'prov':'Bizkaia',
                  'cod_est':'1082'},
'Alavés': {'prov':'Araba',
           'cod_est':'9091R'},
'Osasuna': {'prov':'Navarra',
            'cod_est':'9263D'},
'Deportivo de La Coruña': {'prov':'A Coruña',
                           'cod_est':'1387'},
'Sevilla': {'prov':'Sevilla',
            'cod_est':'5783'},
'Real Madrid': {'prov':'Madrid',
                'cod_est':'3195'},
'Leganés': {'prov':'Madrid',
            'cod_est':'3195'},
'Levante': {'prov':'Valencia',
            'cod_est':'8414A'},
'Barcelona': {'prov':'Barcelona',
              'cod_est':'0076'},
'Getafe': {'prov':'Madrid',
           'cod_est':'3195'},
'Málaga': {'prov':'Málaga',
           'cod_est':'6156X'},
'Villarreal': {'prov':'Castellón',
               'cod_est':'8500A'},
'Atlético de Madrid': {'prov':'Madrid',
                       'cod_est':'3195'},
'Real Sociedad': {'prov':'Gipuzkoa',
                  'cod_est':'1014A'},
'Las Palmas': {'prov':'Las Palmas',
               'cod_est':'C659M'},
'Celta de Vigo': {'prov':'Pontevedra',
         'cod_est':'1495'},
'Betis': {'prov':'Sevilla',
          'cod_est':'5783'},
'Valencia': {'prov':'Valencia',
             'cod_est':'8414A'}

}
